In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import copy
import plotly.express as px
import plotly.io as pio

#Pre-Processing
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics

#models
from sklearn.cluster import KMeans
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
!pip install xgboost
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

#Kernels
from sklearn.gaussian_process.kernels import RBF, WhiteKernel,Matern,ExpSineSquared,RationalQuadratic, ConstantKernel as C

#Scipy
from scipy.spatial import distance
import scipy
import scipy.optimize as opt
from scipy.optimize import differential_evolution
from scipy.optimize import basinhopping
from scipy.optimize import minimize
from scipy.stats import norm

#Pytorch
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
from torch import optim

#For plots
!pip install kaleido
!sudo apt-get install poppler-utils

#Optuna
!pip install optuna
import optuna


In [ ]:
!git clone https://github.com/ihdavjar/LCBM_analysis.git

**Data Preparation for the Lefse**

In [ ]:
data_1=pd.read_csv("/content/LCBM_analysis/feature-table.tsv")
data_1

In [ ]:
org_data=pd.read_csv("/content/LCBM_analysis/feature-table.tsv")

taxonomy_arr=list(org_data["taxonomy"])
otu_ids=list(org_data["otu_id"])
print(taxonomy_arr)
print(otu_ids)

#Removing the taxonomy and the otu_ids from the feature table
org_data=org_data.drop(["otu_id","taxonomy"],axis=1)


temp_data=np.array(org_data)
temp_otus=[]
for i in range(len(temp_data)):
  temp_otus.append("Otu"+str(i+1))


#Adding the labels to feature table
org_data=org_data.T
org_data.columns=temp_otus

print(org_data)
temp_data=pd.read_csv("/content/LCBM_analysis/label_data.csv")
temp_data=temp_data.drop(['Unnamed: 0'],axis=1)
temp_data_np=np.array(temp_data)

print(temp_data)

dict_temp={}
for i in range(len(temp_data_np)):
  if (temp_data_np[i,1]=="Squamous cell carcinoma"):
    dict_temp[temp_data_np[i,0]]="Squamous"
  else:
    dict_temp[temp_data_np[i,0]]=temp_data_np[i,1]

l_temp_row=list(org_data.index)
temp_labels=[]

for i in range(len(l_temp_row)):
  if (dict_temp[l_temp_row[i]]=="Adenosquamous carcinoma"):
    org_data.drop([l_temp_row[i]],axis=0,inplace=True)
  else:
    temp_labels.append(dict_temp[l_temp_row[i]])


org_data_np=np.array(org_data)
org_data

# org_data.insert(loc=0,column="label",value=0.03)
# org_data.insert(loc=1,column="Group",value=list(org_data.index))
# org_data.insert(loc=2,column="numOtus",value=3607)
org_data['treatment']=temp_labels


org_data.to_csv('complete_otus.csv',index=False)
# files.download('complete_otus.csv')
org_data


**Data Standardisation**

In [ ]:
le = preprocessing.LabelEncoder()

org_data_np=np.array(org_data)

labels=le.fit_transform(org_data_np[:,len(org_data_np[0])-1])

org_data_np[:,len(org_data_np[0])-1]=labels

stan_data=pd.DataFrame(org_data_np,columns=org_data.columns,index=org_data.index)
stan_data

In [ ]:
l_cols=taxonomy_arr
print(l_cols)
print(len(l_cols))
for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  print(temp)

**Phylum Wise Division**

In [ ]:
phyllum_labels=[]
count_no_phyllum=0

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)!=1):
    if (temp[1] not in phyllum_labels):
      phyllum_labels.append(temp[1])

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)==1):
    count_no_phyllum=count_no_phyllum+1

print("Number of Taxa without the phyllum labels:-",count_no_phyllum)

print("No. of distinct phyllum are:-",len(phyllum_labels))
print(phyllum_labels)

In [ ]:
temp_dict={}
for i in range(len(phyllum_labels)):
  temp_dict[phyllum_labels[i]]=0

temp_dict

In [ ]:
numpy_data=np.array(stan_data)
print(l_cols)
stan_data

In [ ]:
temps_arr=[]

for i in range(len(numpy_data)):
  temp_arr=numpy_data[i,0:len(numpy_data[0])-1].ravel()
  new_dict=copy.deepcopy(temp_dict)
  for j in range(len(temp_arr)):
    temp=l_cols[j]
    temp=temp.split(";")
    if (len(temp)!=1):
      new_dict[temp[1]]=new_dict[temp[1]]+temp_arr[j]
  temps_arr.append(list(new_dict.values()))

temps_arr=np.array(temps_arr)

temp_columns=list(temp_dict.keys())

phylum_data=pd.DataFrame(temps_arr,columns=np.array(temp_columns),index=np.array(stan_data.index))
phylum_data['treatment']=list(stan_data["treatment"])
print(temps_arr)

In [ ]:
phylum_data

**Lefse Data**

In [ ]:
data_1=pd.read_csv("/content/LCBM_analysis/phylum_data.csv")
data_1.dropna(inplace=True)
data_1.drop(["Unnamed: 0"],axis=1,inplace=True)
data_1

In [ ]:
data_np=np.array(data_1)
temp_labels=[]
for i in range(len(data_1)):
  if (data_np[i,1]<0):
    temp_labels.append("Adenocarcinoma")
  else:
    temp_labels.append("Squamous")

data_1["Class"]=temp_labels

temp_data.append(data_1)
data_1

In [ ]:
import plotly.express as px
# data_canada = px.data.gapminder().query("country == 'Canada'")
fig = px.bar(data_1, x='scores', y='Names',orientation='h',color="Class",labels=dict(Names="", scores="LDA Score", Class="Groups"))

fig.show()

In [ ]:
data_1=pd.read_csv("/content/LCBM_analysis/phylum_data.csv",sep=",")
data_1.dropna(inplace=True)
data_1.drop(["Unnamed: 0"],axis=1,inplace=True)

data_1 = data_1[abs(data_1['scores'])>1]
data_1_np=np.array(data_1)
data_1

In [ ]:
chosen_phylum=data_1_np[:,0]


l = chosen_phylum
final_list = []
for i in l:
  final_list.append(i.strip())

chosen_phylum=final_list
print("Statistically significant phylum are:\n",chosen_phylum)

temp_phylums_names=[]
for i in range(len(chosen_phylum)):
  temp_phylums_names.append(" "+str(chosen_phylum[i]))

chosen_columns1=phylum_data.loc[:,temp_phylums_names]

chosen_columns1

**Genus Wise Division**

In [ ]:
genus_labels=[]
l_cols=taxonomy_arr

for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)>5):
    if (temp[5] not in genus_labels):
      genus_labels.append(temp[5])

count_no_phyllum=0
for i in range(len(l_cols)):
  temp=l_cols[i]
  temp=temp.split(";")
  if (len(temp)==5):
    count_no_phyllum=count_no_phyllum+1

print("Number of Taxa without the Genus labels:-",count_no_phyllum)

print("No. of distinct Genus:-",len(genus_labels))
print(genus_labels)

In [ ]:
temp_dict={}
for i in range(len(genus_labels)):
  temp_dict[genus_labels[i]]=0

temp_dict

In [ ]:
numpy_data=np.array(stan_data)
print(l_cols)
stan_data

In [ ]:
temps_arr=[]
for i in range(len(numpy_data)):

  temp_arr=numpy_data[i,0:len(numpy_data[0])-1].ravel()
  new_dict=copy.deepcopy(temp_dict)
  for j in range(len(temp_arr)):
    temp=l_cols[j]
    temp=temp.split(";")
    if (len(temp)>5):
      new_dict[temp[5]]=new_dict[temp[5]]+temp_arr[j]
  temps_arr.append(list(new_dict.values()))

temps_arr=np.array(temps_arr)
genus_data=pd.DataFrame(temps_arr,columns=list(temp_dict.keys()),index=org_data.index)
genus_data['treatment']=list(stan_data["treatment"])
print(temps_arr)

In [ ]:
genus_data

**lefse data**

In [ ]:
data_1=pd.read_csv("/content/LCBM_analysis/genus_data.csv")
data_1.dropna(inplace=True)
data_1.drop(["Unnamed: 0"],axis=1,inplace=True)
data_1

In [ ]:
data_np=np.array(data_1)
temp_labels=[]
for i in range(len(data_1)):
  if (data_np[i,1]<0):
    temp_labels.append("Adenocarcinoma")
  else:
    temp_labels.append("Squamous")

data_1["Class"]=temp_labels

temp_data.append(data_1)

data_1

In [ ]:
import plotly.express as px
# data_canada = px.data.gapminder().query("country == 'Canada'")
fig = px.bar(data_1, x='scores', y='Names',orientation='h',color="Class",labels=dict(Names="", scores="LDA Score", Class="Groups"))

fig.show()

In [ ]:
data_2=pd.read_csv("/content/LCBM_analysis/genus_data.csv",sep=",")
data_2.dropna(inplace=True)
data_2.drop(["Unnamed: 0"],axis=1,inplace=True)

data_2 = data_2[abs(data_2['scores'])>1]
data_2_np=np.array(data_2)
data_2

In [ ]:
chosen_genus=data_2_np[:,0]


l = chosen_genus
final_list = []
for i in l:
  final_list.append(i.strip())

chosen_genus=final_list
print("Statistically significant genus are:\n",chosen_genus)

temp_genus_names=[]
for i in range(len(chosen_genus)):
  if (str(chosen_genus[i])=='g__0319.6G20'):
    temp_genus_names.append(" g__0319-6G20")
  else:
    temp_genus_names.append(" "+str(chosen_genus[i]))

chosen_columns2=genus_data.loc[:,temp_genus_names]

chosen_columns2

**Collecting Useful Columns from the MetaData**

In [ ]:
meta_data=pd.read_csv("/content/LCBM_analysis/SraRunTable.tsv",sep='\t')
meta_data.drop([0],axis=0,inplace=True)
meta_data

**Choosing the important columns**

In [ ]:
print(meta_data.columns)

**Choosing env_material, Host_Age, host_sex, smoker**

In [ ]:
meta_data=meta_data[[" Sample-id","env_material","host_sex","Host_Age","smoker"]]
meta_data

In [ ]:
meta_data_np=np.array(meta_data)
print(meta_data_np)

temp_sample_dict={}

for i in range(len(meta_data_np)):
  temp_sample_dict[meta_data_np[i,0]]=meta_data_np[i,1:]

In [ ]:
temp_sample_dict

In [ ]:
temp_sample_names=list(phylum_data.index)

chosen_column3_np=[]

for i in range(len(temp_sample_names)):
  chosen_column3_np.append(temp_sample_dict[temp_sample_names[i]])

chosen_column3_np=np.array(chosen_column3_np)
chosen_column3_np

chosen_columns3=pd.DataFrame(chosen_column3_np,index=temp_sample_names,columns=list(meta_data.columns)[1:])
chosen_columns3

**Removing Instances with missing Data**

In [ ]:
meta_data.dropna(inplace=True)
meta_data

**Data Preprocessing**

In [ ]:
chosen_column1_np=np.array(chosen_columns1)
chosen_column2_np=np.array(chosen_columns2)
chosen_column3_np=np.array(chosen_columns3)

data=np.array(stan_data)

x_data=np.c_[chosen_column1_np,chosen_column2_np,chosen_column3_np]
print(np.shape(x_data))

y_data=np.array(stan_data["treatment"]).astype(int)

result_data=pd.DataFrame(x_data,index=phylum_data.index,columns=list(chosen_columns1.columns)+list(chosen_columns2.columns)+list(chosen_columns3.columns))
result_data["label"]=y_data


In [ ]:
# result_data.to_csv("data_dnn.csv")
# files.download("data_dnn.csv")

**label encoding**

In [ ]:
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()

result_data["env_material"]=le1.fit_transform(result_data["env_material"])
result_data["host_sex"]=le2.fit_transform(result_data["host_sex"])
result_data["smoker"]=le3.fit_transform(result_data["smoker"])

# result_data.drop(['env_material','Host_Age','host_sex'],axis=1,inplace=True)
result_data

In [ ]:
result_data

In [ ]:
def bray_curtis_array(l1,l2):
  bc=distance.braycurtis(l1.astype(float),l2.astype(float))
  return bc


def bray_curtis_comp(data):
  d_mat=[]

  for i in range(len(data)):
    d_mat1=[]
    for j in range(len(data)):
      temp_1=data[i,:len(data[0])-1]
      temp_2=data[j,:len(data[0])-1]
      d_mat1.append(bray_curtis_array(temp_1,temp_2))
    d_mat.append(d_mat1)
  d_mat=np.array(d_mat)
  return d_mat

result_data_out_np=np.array(result_data)
bray_curtis_data=pd.DataFrame(bray_curtis_comp(result_data_out_np))


In [ ]:
bray_curtis_data_np=np.array(bray_curtis_data)

X = bray_curtis_data_np
pca = PCA(n_components=4)
pca.fit(X)

print(pca.explained_variance_ratio_)

x_new=pca.transform(X)

x=x_new[:,0]
y=x_new[:,1]
z=x_new[:,2]
z1=x_new[:,3]

In [ ]:
fig = px.scatter(x=x, y=y,color=org_data['treatment'])
fig.show()

In [ ]:
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from plotly.offline import plot, iplot, init_notebook_mode
from plotly.subplots import make_subplots
# df = px.data.iris()
fig = px.scatter_3d(x=x, y=y, z=z,
              color=org_data['treatment'])



# pio.write_image(fig, 'image.png',scale=7, width=1080, height=1080)
# files.download('image.png')

fig.show()

**Model Building**

In [ ]:
result_data_no_out = copy.deepcopy(result_data)
x_data_pd=result_data_no_out.drop(['label'],axis=1)

#Normalisation
scaler = preprocessing.MinMaxScaler()

x_data_pd_stan=pd.DataFrame(scaler.fit_transform(x_data_pd),columns=x_data_pd.columns,index=x_data_pd.index)

x_data_pd_stan= x_data_pd

x_data_pd_stan["sample_id"] = result_data.index

x_data_pd['label']=result_data['label']

x_data_pd_stan['label']=result_data['label']


result_data=x_data_pd_stan


result_data_class0=result_data[result_data["label"]==0]
result_data_class1=result_data[result_data["label"]==1]

result_data_class_0_np=np.array(result_data_class0)
result_data_class_1_np=np.array(result_data_class1)

print(np.unique(result_data_class_0_np[:,len(result_data_class_0_np[0])-1]))
print(np.unique(result_data_class_1_np[:,len(result_data_class_1_np[0])-1]))

x_class_0=result_data_class_0_np[:,0:len(result_data_class_0_np[0])-1]
y_class_0=result_data_class_0_np[:,len(result_data_class_0_np[0])-1].astype(int)

x_class_1=result_data_class_1_np[:,0:len(result_data_class_1_np[0])-1]
y_class_1=result_data_class_1_np[:,len(result_data_class_1_np[0])-1].astype(int)

#CLASS 0
x_train_0, x_temp_0, y_train_0, y_temp_0 = train_test_split(x_class_0, y_class_0, test_size=0.3,shuffle=True, random_state=42)
x_val_0, x_test_0, y_val_0, y_test_0 = train_test_split(x_temp_0, y_temp_0, test_size=0.5,shuffle=True, random_state=42)

#CLASS 1
x_train_1, x_temp_1, y_train_1, y_temp_1 = train_test_split(x_class_1, y_class_1, test_size=0.3,shuffle=True, random_state=42)
x_val_1, x_test_1, y_val_1, y_test_1 = train_test_split(x_temp_1, y_temp_1, test_size=0.5,shuffle=True, random_state=42)

#Train Data
x_train=np.r_[x_train_0,x_train_1]
y_train=np.r_[y_train_0,y_train_1]

sample_id_train = x_train[:,len(x_train[0])-1]

x_train = x_train[:,:len(x_train[0])-1].astype(float)


#Val Data
x_val=np.r_[x_val_0,x_val_1]
y_val=np.r_[y_val_0,y_val_1]

sample_id_val = x_val[:,len(x_val[0])-1]

x_val = x_val[:,:len(x_val[0])-1].astype(float)

#Test Data
x_test=np.r_[x_test_0,x_test_1]
y_test=np.r_[y_test_0,y_test_1]

sample_id_test = x_test[:,len(x_test[0])-1]

x_test = x_test[:,:len(x_test[0])-1].astype(float)


print("Number of Instances in Training Data of class 0:-",len(x_train_0))
print("Number of Instances in Training Data of class 1:-",len(x_train_1))

print("Number of Instances in Validation Data of class 0:-",len(x_val_0))
print("Number of Instances in Validation Data of class 1:-",len(x_val_1))

print("Number of Instances in Testing Data of class 0:-",len(x_test_0))
print("Number of Instances in Testing Data of class 1:-",len(x_test_1))



#Test Full Data
x_test_1=np.r_[x_temp_0,x_temp_1]
y_test_1=np.r_[y_temp_0,y_temp_1]

sample_id_test_1 = x_test_1[:,len(x_test_1[0])-1]

x_test_1 = x_test_1[:,:len(x_test_1[0])-1].astype(float)

print("Number of Instance in Testing Data 1 of class 0:-",len(x_temp_0))
print("Number of Instance in Testing Data 1 of class 1:-",len(x_temp_1))


**Downloading The Data**

In [ ]:
l_cols_stan = list(x_data_pd_stan.columns)

l_cols_stan = l_cols_stan[0:len(l_cols_stan)-1]

x_train_pd = pd.DataFrame(x_train,columns=l_cols_stan[:len(l_cols_stan)-1])

x_train_pd["label"] = y_train

x_train_pd.to_csv("Training_data.csv")

x_test_pd = pd.DataFrame(x_test_1,columns=l_cols_stan[:len(l_cols_stan)-1])

x_test_pd["label"] = y_test_1

x_test_pd.to_csv("Testing_data.csv")

**Model Training**

**Logistic Regression**

In [ ]:
model_lr = LogisticRegression(random_state=0,max_iter=1000).fit(x_train,y_train)
y_pred_lr=model_lr.predict(x_test)

print("Training Accuracy is:-",model_lr.score(x_train,y_train))
print("Classification Accuracy is:-",model_lr.score(x_test,y_test))

In [ ]:
temp_arr=model_lr.coef_
print(temp_arr)
temp_arr.sort()
temp_arr

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_lr, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_lr,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_lr.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Gaussian Naive Bayes**

In [ ]:
model_nb = GaussianNB().fit(x_train,y_train)
y_pred_nb=model_nb.predict(x_test)

print("Training Accuracy is:-",model_nb.score(x_train,y_train))
print("Classification Accuracy is:-",model_nb.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_nb, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_nb,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_nb.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Decision Tree Classifier**

In [ ]:
acc=[]
for i in range(20):
  model = DecisionTreeClassifier(max_depth=i+1).fit(x_train,y_train)
  acc.append(model.score(x_val,y_val))

plt.plot(acc)
plt.show()
best_depth=np.argmax(acc)+1
print(best_depth)

In [ ]:
model_dc = DecisionTreeClassifier(max_depth=best_depth).fit(x_train,y_train)
y_pred_dc=model_dc.predict(x_test)

print("Training Accuracy is:-",model_dc.score(x_train,y_train))
print("Classification Accuracy is:-",model_dc.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_dc, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_dc,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_dc.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Random Forest Classifier**

In [ ]:
acc=[]
for i in range(20):
  model = RandomForestClassifier(max_depth=i+1,n_estimators=1000).fit(x_train,y_train)
  acc.append(model.score(x_val,y_val))

plt.plot(acc)
plt.show()
best_depth=np.argmax(acc)+1
print(best_depth)

In [ ]:
model_rfc = RandomForestClassifier(max_depth=best_depth,n_estimators=1000,random_state=0).fit(x_train,y_train)
y_pred_rfc=model_rfc.predict(x_test)


print("Training Accuracy is:-",model_rfc.score(x_train,y_train))
print("Classification Accuracy is:-",model_rfc.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_rfc, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_rfc,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_rfc.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**XGBoost Classifier**

In [ ]:
acc=[]
for i in range(10):
  model = XGBClassifier(max_depth=i+1).fit(x_train,y_train)
  acc.append(model.score(x_val,y_val))

plt.plot(acc)
plt.show()
best_depth=np.argmax(acc)+1
print(best_depth)

In [ ]:
model_xgb = XGBClassifier(max_depth=best_depth,random_state=0).fit(x_train,y_train)
y_pred_xgb=model_xgb.predict(x_test)

print("Training Accuracy is:-",model_xgb.score(x_train,y_train))
print("Classification Accuracy is:-",model_xgb.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_xgb, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_xgb,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_xgb.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Gaussian Process Classifier**

In [ ]:
lbound = 1e-5
rbound = 1e5

n_restarts = 50

n_features = len(x_train[0])

kernel=C(1.0, (lbound,rbound)) * RBF(n_features*[n_features], (lbound,rbound))  + WhiteKernel()

model_gpc = GaussianProcessClassifier().fit(x_train,y_train)
y_pred_gpc=model_gpc.predict(x_test)

print("Classification Accuracy is:-",model_gpc.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_gpc, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_gpc,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_gpc.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**KNN Classifier**

In [ ]:
acc=[]
for i in range(30):
  model = KNeighborsClassifier(n_neighbors=i+1).fit(x_train,y_train)
  acc.append(model.score(x_val,y_val))

plt.plot(acc)
plt.show()
best_n=np.argmax(acc)+1
print(best_n)

In [ ]:
model_knn = KNeighborsClassifier(n_neighbors=best_n).fit(x_train,y_train)
y_pred_knn=model_knn.predict(x_test)

print("Classification Accuracy is:-",model_knn.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_knn, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_knn,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_knn.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Neural Network Classifier**

In [ ]:
#For the Standardised data
acc1=[]
for i in range(1):
  model = MLPClassifier(random_state=1,hidden_layer_sizes=[i+1], max_iter=3000)
  model.fit(x_train,y_train.ravel())
  acc1.append(model.score(x_val,y_val.ravel()))

plt.plot(acc1)
plt.show()

best_hidden1=np.argmax(acc1)+1

In [ ]:
model_nn = MLPClassifier(random_state=1,hidden_layer_sizes=[best_hidden1], max_iter=3000).fit(x_train,y_train)
y_pred_nn=model_nn.predict(x_test)

print(best_hidden1)

print("Accuracy on the training data is:-",model_nn.score(x_train,y_train))
print("Classification Accuracy is:-",model_nn.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_nn, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_nn,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_nn.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**LDA Classifier**

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model_lda =LinearDiscriminantAnalysis().fit(x_train,y_train)
y_pred_lda=model_lda.predict(x_test)

print("Classification Accuracy is:-",model_lda.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_lda, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_lda,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_lda.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Support Vector Classifier**

In [ ]:
from sklearn.svm import SVC

model_svc = SVC(probability=True,kernel='rbf').fit(x_train,y_train)
y_pred_svc=model_svc.predict(x_test)

print("Classification Accuracy is:-",model_svc.score(x_test,y_test))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test, y_pred_svc, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_svc,y_test)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_svc.predict_proba(x_test)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Sk-learn Neural Network Optimisation**

In [ ]:
def objective(trial):
  n_layers = trial.suggest_int("n_layers", 1, 5)

  temp_neurons=[]

  for i in range(n_layers):
    temp_neurons.append(trial.suggest_int(str(i), 1, 1000))


  model = MLPClassifier(random_state=1,hidden_layer_sizes=temp_neurons, max_iter=3000)

  #Training the Model

  model.fit(x_train,y_train.ravel())

  y_pred = model.predict(x_val)

  v0 = metrics.f1_score(y_val, y_pred, average='micro')

  return v0


study = optuna.create_study(directions=["maximize"])
study.optimize(objective, n_trials=500)

print(study.best_params)

In [ ]:
print(np.array(list(study.best_params.values()))[1:].astype(int))
temp_arrr=np.array(list(study.best_params.values()))[1:].astype(int)

In [ ]:
model_nn = MLPClassifier(random_state=1,hidden_layer_sizes=temp_arrr, max_iter=3000).fit(x_train,y_train)
y_pred_nn=model_nn.predict(x_test_1)

print("Accuracy on the training data is:-",model_nn.score(x_train,y_train))
print("Classification Accuracy is:-",model_nn.score(x_test_1,y_test_1))

In [ ]:
target_names = ['Adenocarcinoma','Squamous']
print(classification_report(y_test_1, y_pred_nn, target_names=target_names))

In [ ]:
dat_lr=confusion_matrix(y_pred_nn,y_test_1)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
probs = model_nn.predict_proba(x_test_1)
preds = probs[:,1]

fpr, tpr, threshold = metrics.roc_curve(y_test_1, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Pytorch Neural Network Implementation**

In [ ]:
x_data=x_train
x_train_T = torch.Tensor(x_train)
y_train=y_train.reshape((-1,1))
y_train_T = torch.Tensor(y_train.astype(int))

train_data = torch.utils.data.TensorDataset(x_train_T,y_train_T)

**Validation Data**

In [ ]:
x_val_T = torch.Tensor(x_val)
y_val=y_val.reshape((-1,1))
y_val_T = torch.Tensor(y_val.astype(int))


**Testing Data**

In [ ]:
x_test_T = torch.Tensor(x_test_1)
y_test_1=y_test_1.reshape((-1,1))
y_test_T = torch.Tensor(y_test_1.astype(int))

test_data = torch.utils.data.TensorDataset(x_test_T,y_test_T)

In [ ]:
train_data_load=torch.utils.data.DataLoader(train_data,batch_size=16,shuffle=True)

**Model Building**

**Two Layers**

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Net2(torch.nn.Module):
    def __init__(self, num_inputs, size_hidden_1, size_hidden_2, n_output):
      super(Net2, self).__init__()

      self.hidden_layer_1 = torch.nn.Linear(num_inputs, size_hidden_1)
      self.activation_hidden_layer_1 = torch.nn.ReLU()

      self.hidden_layer_2 = torch.nn.Linear(size_hidden_1, size_hidden_2)
      self.activation_hidden_layer_2 = torch.nn.ReLU()

      self.dropout = nn.Dropout(p=0.3)

      self.output_layer = torch.nn.Linear(size_hidden_2, n_output)
      self.activation_output_layer = torch.nn.Sigmoid()


    def forward(self, x):

      temp_x1 = self.activation_hidden_layer_1(self.hidden_layer_1(x))

      temp_x2 = self.activation_hidden_layer_2(self.hidden_layer_2(temp_x1))

      temp_x3 = self.dropout(temp_x2)

      temp_x4 = self.activation_output_layer(self.output_layer(temp_x3))

      return temp_x4

**Three Layers**

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Net1(torch.nn.Module):
    def __init__(self, num_inputs, size_hidden_1, size_hidden_2,size_hidden_3 , n_output):
      super(Net1, self).__init__()

      self.hidden_layer_1 = torch.nn.Linear(num_inputs, size_hidden_1)
      self.activation_hidden_layer_1 = torch.nn.ReLU()

      self.hidden_layer_2 = torch.nn.Linear(size_hidden_1, size_hidden_2)
      self.activation_hidden_layer_2 = torch.nn.ReLU()

      self.hidden_layer_3 = torch.nn.Linear(size_hidden_2, size_hidden_3)
      self.activation_hidden_layer_3 = torch.nn.ReLU()

      self.dropout = nn.Dropout(p=0.3)

      self.output_layer = torch.nn.Linear(size_hidden_3, n_output)
      self.activation_output_layer = torch.nn.Sigmoid()


    def forward(self, x):

      temp_x1 = self.activation_hidden_layer_1(self.hidden_layer_1(x))

      temp_x2 = self.activation_hidden_layer_2(self.hidden_layer_2(temp_x1))

      temp_x3 = self.activation_hidden_layer_3(self.hidden_layer_3(temp_x2))

      temp_x4 = self.dropout(temp_x3)

      temp_x5 = self.activation_output_layer(self.output_layer(temp_x4))

      return temp_x5

**Four Layers**

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Net3(torch.nn.Module):
    def __init__(self, num_inputs, size_hidden_1, size_hidden_2,size_hidden_3 ,size_hidden_4 , n_output):
      super(Net3, self).__init__()

      self.hidden_layer_1 = torch.nn.Linear(num_inputs, size_hidden_1)
      self.activation_hidden_layer_1 = torch.nn.ReLU()

      self.hidden_layer_2 = torch.nn.Linear(size_hidden_1, size_hidden_2)
      self.activation_hidden_layer_2 = torch.nn.ReLU()

      self.hidden_layer_3 = torch.nn.Linear(size_hidden_2, size_hidden_3)
      self.activation_hidden_layer_3 = torch.nn.ReLU()

      self.hidden_layer_4 = torch.nn.Linear(size_hidden_3, size_hidden_4)
      self.activation_hidden_layer_4 = torch.nn.ReLU()

      self.dropout = nn.Dropout(p=0.3)

      self.output_layer = torch.nn.Linear(size_hidden_4, n_output)
      self.activation_output_layer = torch.nn.Sigmoid()


    def forward(self, x):

      temp_x1 = self.activation_hidden_layer_1(self.hidden_layer_1(x))

      temp_x2 = self.activation_hidden_layer_2(self.hidden_layer_2(temp_x1))

      temp_x3 = self.activation_hidden_layer_3(self.hidden_layer_3(temp_x2))

      temp_x4 = self.activation_hidden_layer_4(self.hidden_layer_4(temp_x3))

      temp_x5 = self.dropout(temp_x4)

      temp_x6 = self.activation_output_layer(self.output_layer(temp_x5))

      return temp_x6

**Hyperparameter Optimisation**

In [ ]:
def objective(trial):
  x = trial.suggest_int("x", 1, 1000)
  y = trial.suggest_int("y", 1, 1000)
  z = trial.suggest_int("z", 1, 1000)

  train_data_load=torch.utils.data.DataLoader(train_data,batch_size=16,shuffle=True)

  model1 = Net1(len(x_data[0]),x,y,z,1)


  #Training the Model
  optimiser=optim.Adam(model1.parameters(),lr=0.001)

  #Loss function
  loss=torch.nn.BCELoss()

  num_epoch=100

  for i in range(num_epoch):
    losses=[]
    for batch in train_data_load:
      x,y=batch

      b=len(x)
      x=x.reshape(b,-1)

      l=model1(x)

      J=loss(l,y)

      optimiser.zero_grad()

      J.backward()

      optimiser.step()

  y_pred = model1(x_val_T)

  y_pred_np = y_pred.detach().numpy()

  y_val_np = y_val_T.numpy()

  y_temp_pred =[1 if value >= 0.5 else 0 for value in y_pred_np]

  print(y_temp_pred)

  v0 = metrics.f1_score(y_val_np, y_temp_pred, average='micro')

  return v0


study = optuna.create_study(directions=["maximize"])
study.optimize(objective, n_trials=250)

print(study.best_params)


In [ ]:
temp_neuron_size = list(study.best_params.values())

model1=Net1(len(x_train[0]),temp_neuron_size[0],temp_neuron_size[1],temp_neuron_size[2],1)

print(model1)

**Training**

In [ ]:
#optimiser
optimiser=optim.Adam(model1.parameters(),lr=0.0005)
# optimiser=optim.SGD(model.parameters(),lr=0.0005,momentum=0.5)

#Loss function
loss=torch.nn.BCELoss()

# loss=torch.nn.L1Loss()

num_epoch=100
loss_epoch_train=[]

loss_epoch_val=[]

for i in range(num_epoch):
  losses=[]
  for batch in train_data_load:
    x,y=batch

    b=len(x)
    x=x.reshape(b,-1)

    l=model1(x)

    J=loss(l,y)

    optimiser.zero_grad()

    J.backward()

    optimiser.step()

    losses.append(J.item())

  y_pred_T=model1(x_val_T)

  J1=loss(y_val_T,y_pred_T)
  loss_epoch_val.append(J1.item())

  print("Training Loss for epoch "+str(i+1)+":",torch.tensor(losses).mean().item())
  print("Validation Loss for epoch "+str(i+1)+":",J1.item())
  loss_epoch_train.append(torch.tensor(losses).mean().item())


plt.plot(loss_epoch_train,label="Train Data")
plt.legend()
plt.xlabel("Number of Epochs")
plt.ylabel("Mean Loss")
plt.show()

plt.plot(loss_epoch_val,label="Val Data")
plt.legend()
plt.xlabel("Number of Epochs")
plt.ylabel("Mean Loss")
plt.show()

In [ ]:
y_pred=model1(x_test_T)
J=loss(y_pred.reshape(1,-1),y_test_T.reshape(1,-1))

print(J.item())

print(y_pred)

y_pred=y_pred.detach().numpy()
y_pred_temp=[]

for i in range(len(y_pred)):
  if (y_pred[i]>=0.5):
    y_pred_temp.append(1)
  else:
    y_pred_temp.append(0)

In [ ]:
y_test_temp=y_test_T.detach().numpy().astype(int)

target_names = ['class 0', 'class 1']

print(classification_report(y_test_temp, y_pred_temp, target_names=target_names))

In [ ]:
print(list(y_test_temp.ravel()))
print(y_pred_temp)

In [ ]:
class_0=[]
class_1=[]
y_pred=y_pred.ravel()
for i in range(len(y_pred_temp)):
  if (y_test_temp[i]==0):
    class_0.append(y_pred[i])
  else:
    class_1.append(y_pred[i])

In [ ]:
plt.hist(class_0,color="red",alpha=0.5)
plt.hist(class_1,color="blue",alpha=0.5)

plt.show()


In [ ]:
dat_lr=confusion_matrix(y_pred_temp,y_test_temp)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
for i in range(len(y_pred_temp)):
  if (y_pred_temp[i]==0 and y_pred_temp[i]!=y_test_temp[i]):
    print(sample_id_test_1[i])

In [ ]:
for i in range(len(y_pred_temp)):
  if (y_pred_temp[i]==1 and y_pred_temp[i]!=y_test_temp[i]):
    print(sample_id_test_1[i])

In [ ]:
preds = y_pred

fpr, tpr, threshold = metrics.roc_curve(y_test_temp, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
def objective(trial):
  x = trial.suggest_int("x", 1, 1000)
  y = trial.suggest_int("y", 1, 1000)

  train_data_load=torch.utils.data.DataLoader(train_data,batch_size=16,shuffle=True)

  model2 = Net2(len(x_data[0]),x,y,1)


  #Training the Model

  optimiser=optim.Adam(model2.parameters(),lr=0.001)

  #Loss function
  loss=torch.nn.BCELoss()

  num_epoch=100

  for i in range(num_epoch):
    losses=[]
    for batch in train_data_load:
      x,y=batch

      b=len(x)
      x=x.reshape(b,-1)

      l=model2(x)

      J=loss(l,y)

      optimiser.zero_grad()

      J.backward()

      optimiser.step()


  y_pred = model2(x_val_T)

  y_pred_np = y_pred.detach().numpy()

  y_val_np = y_val_T.numpy()

  y_temp_pred = y_pred_np>0.5

  v0 = metrics.f1_score(y_val_np, y_temp_pred, average='micro')

  return v0


study = optuna.create_study(directions=["maximize"])
study.optimize(objective, n_trials=200)

print(study.best_params)


In [ ]:
temp_neuron_size = list(study.best_params.values())

model2=Net2(len(x_train[0]),temp_neuron_size[0],temp_neuron_size[1],1)

print(model2)

**Training**

In [ ]:
#optimiser
optimiser=optim.Adam(model2.parameters(),lr=0.0005)
# optimiser=optim.SGD(model.parameters(),lr=0.0005,momentum=0.5)

#Loss function
loss=torch.nn.BCELoss()

num_epoch=100
loss_epoch_train=[]

loss_epoch_val=[]

for i in range(num_epoch):
  losses=[]
  for batch in train_data_load:
    x,y=batch

    b=len(x)
    x=x.reshape(b,-1)

    l=model2(x)

    J=loss(l,y)

    optimiser.zero_grad()

    J.backward()

    optimiser.step()

    losses.append(J.item())

  y_pred_T=model2(x_val_T)

  J1=loss(y_val_T,y_pred_T)
  loss_epoch_val.append(J1.item())

  print("Training Loss for epoch "+str(i+1)+":",torch.tensor(losses).mean().item())
  print("Validation Loss for epoch "+str(i+1)+":",J1.item())
  loss_epoch_train.append(torch.tensor(losses).mean().item())


plt.plot(loss_epoch_train,label="Train Data")
plt.legend()
plt.xlabel("Number of Epochs")
plt.ylabel("Mean Loss")
plt.show()

plt.plot(loss_epoch_val,label="Val Data")
plt.legend()
plt.xlabel("Number of Epochs")
plt.ylabel("Mean Loss")
plt.show()

In [ ]:
y_pred=model2(x_test_T)
J=loss(y_pred.reshape(1,-1),y_test_T.reshape(1,-1))

print(J.item())

print(y_pred)

y_pred=y_pred.detach().numpy()
y_pred_temp=[]

for i in range(len(y_pred)):
  if (y_pred[i]>0.4):
    y_pred_temp.append(1)
  else:
    y_pred_temp.append(0)

In [ ]:
y_test_temp=y_test_T.detach().numpy().astype(int)

target_names = ['class 0', 'class 1']
print(classification_report(y_test_temp, y_pred_temp, target_names=target_names))

In [ ]:
print(list(y_test_temp.ravel()))
print(y_pred_temp)

In [ ]:
class_0=[]
class_1=[]
y_pred=y_pred.ravel()
for i in range(len(y_pred_temp)):
  if (y_test_temp[i]==0):
    class_0.append(y_pred[i])
  else:
    class_1.append(y_pred[i])

In [ ]:
plt.hist(class_0,color="red",alpha=0.5)
plt.hist(class_1,color="blue",alpha=0.5)

plt.show()


In [ ]:
dat_lr=confusion_matrix(y_pred_temp,y_test_temp)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
for i in range(len(y_pred_temp)):
  if (y_pred_temp[i]==0 and y_pred_temp[i]!=y_test_temp[i]):
    print(sample_id_test_1[i])

In [ ]:
for i in range(len(y_pred_temp)):
  if (y_pred_temp[i]==1 and y_pred_temp[i]!=y_test_temp[i]):
    print(sample_id_test_1[i])

In [ ]:
preds = y_pred

fpr, tpr, threshold = metrics.roc_curve(y_test_temp, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
def objective(trial):
  x = trial.suggest_int("x", 1, 1000)
  y = trial.suggest_int("y", 1, 1000)
  z = trial.suggest_int("z", 1, 1000)
  a = trial.suggest_int("a", 1, 1000)

  train_data_load=torch.utils.data.DataLoader(train_data,batch_size=16,shuffle=True)

  model3 = Net3(len(x_data[0]),x,y,z,a,1)


  #Training the Model
  optimiser=optim.Adam(model3.parameters(),lr=0.001)

  #Loss function
  loss=torch.nn.BCELoss()

  num_epoch=100

  for i in range(num_epoch):
    losses=[]
    for batch in train_data_load:
      x,y=batch

      b=len(x)
      x=x.reshape(b,-1)

      l=model3(x)

      J=loss(l,y)

      optimiser.zero_grad()

      J.backward()

      optimiser.step()

  y_pred = model3(x_val_T)

  y_pred_np = y_pred.detach().numpy()

  y_val_np = y_val_T.numpy()

  y_temp_pred =[1 if value >= 0.5 else 0 for value in y_pred_np]

  print(y_temp_pred)

  v0 = metrics.f1_score(y_val_np, y_temp_pred, average='micro')

  return v0


study = optuna.create_study(directions=["maximize"])
study.optimize(objective, n_trials=250)

print(study.best_params)


In [ ]:
temp_neuron_size = list(study.best_params.values())

model1=Net3(len(x_train[0]),temp_neuron_size[0],temp_neuron_size[1],temp_neuron_size[2],temp_neuron_size[3],1)

print(model1)

**Training**

In [ ]:
#optimiser
optimiser=optim.Adam(model1.parameters(),lr=0.0005)
# optimiser=optim.SGD(model.parameters(),lr=0.0005,momentum=0.5)

#Loss function
loss=torch.nn.BCELoss()

num_epoch=100
loss_epoch_train=[]

loss_epoch_val=[]

for i in range(num_epoch):
  losses=[]
  for batch in train_data_load:
    x,y=batch

    b=len(x)
    x=x.reshape(b,-1)

    l=model1(x)

    J=loss(l,y)

    optimiser.zero_grad()

    J.backward()

    optimiser.step()

    losses.append(J.item())

  y_pred_T=model1(x_val_T)

  J1=loss(y_pred_T,y_val_T)
  loss_epoch_val.append(J1.item())

  print("Training Loss for epoch "+str(i+1)+":",torch.tensor(losses).mean().item())
  print("Validation Loss for epoch "+str(i+1)+":",J1.item())
  loss_epoch_train.append(torch.tensor(losses).mean().item())


plt.plot(loss_epoch_train,label="Train Data")
plt.legend()
plt.xlabel("Number of Epochs")
plt.ylabel("Mean Loss")
plt.show()

plt.plot(loss_epoch_val,label="Val Data")
plt.legend()
plt.xlabel("Number of Epochs")
plt.ylabel("Mean Loss")
plt.show()

In [ ]:
y_pred=model1(x_test_T)
J=loss(y_pred.reshape(1,-1),y_test_T.reshape(1,-1))

print(J.item())

print(y_pred)

y_pred=y_pred.detach().numpy()
y_pred_temp=[]

for i in range(len(y_pred)):
  if (y_pred[i]>=0.5):
    y_pred_temp.append(1)
  else:
    y_pred_temp.append(0)

In [ ]:
y_test_temp=y_test_T.detach().numpy().astype(int)

target_names = ['class 0', 'class 1']

print(classification_report(y_test_temp, y_pred_temp, target_names=target_names))

In [ ]:
print(list(y_test_temp.ravel()))
print(y_pred_temp)

In [ ]:
class_0=[]
class_1=[]
y_pred=y_pred.ravel()
for i in range(len(y_pred_temp)):
  if (y_test_temp[i]==0):
    class_0.append(y_pred[i])
  else:
    class_1.append(y_pred[i])

In [ ]:
plt.hist(class_0,color="red",alpha=0.5)
plt.hist(class_1,color="blue",alpha=0.5)

plt.show()


In [ ]:
dat_lr=confusion_matrix(y_pred_temp,y_test_temp)
sns.heatmap(dat_lr,annot=True,xticklabels=['Adenocarcinoma','Squamous'],
            yticklabels=['Adenocarcinoma','Squamous'])
plt.title("Standardised Data")
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.show()

In [ ]:
for i in range(len(y_pred_temp)):
  if (y_pred_temp[i]==0 and y_pred_temp[i]!=y_test_temp[i]):
    print(sample_id_test_1[i])

In [ ]:
for i in range(len(y_pred_temp)):
  if (y_pred_temp[i]==1 and y_pred_temp[i]!=y_test_temp[i]):
    print(sample_id_test_1[i])

In [ ]:
preds = y_pred

fpr, tpr, threshold = metrics.roc_curve(y_test_temp, preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

**Using Basinhoppin**

In [ ]:
def scoring_fun(x):
  print(x)
  model = Net1(len(x_train[0]),int(x[0]*1000),int(x[1]*1000),int(x[2]*1000),1)
  optimiser=optim.Adam(model1.parameters(),lr=0.001)

  #Loss function
  loss=torch.nn.BCELoss()

  num_epoch=50

  for i in range(num_epoch):
    losses=[]
    for batch in train_data_load:
      x,y=batch

      b=len(x)
      x=x.reshape(b,-1)

      l=model1(x)

      J=loss(l,y)

      optimiser.zero_grad()

      J.backward()

      optimiser.step()

  y_pred = model1(x_val_T)

  y_pred_np = y_pred.detach().numpy()

  y_val_np = y_val_T.numpy()

  y_temp_pred =[1 if value >= 0.5 else 0 for value in y_pred_np]

  print(y_temp_pred)

  v0 = metrics.f1_score(y_val_np, y_temp_pred, average='micro')

  return -1*v0

